In [45]:
import pandas as pd
import os
import numpy as np
import datetime
import glob
import gc

In [8]:
def recursive_file_gen(root_path):
    for root, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root,file)

In [36]:
df_week_start_group=pd.DataFrame({"week_start_Wednesday":[datetime.date(2019,3,20)+datetime.timedelta(days=x*7) for x in range(9)]})
df_week_start_group['week_period']=["P1"]*4+["P2"]*1+["P3"]*1+["P4"]*3

df_group_total_counts=pd.DataFrame({'group':['test_in_test','control_400K','balance'],'total_counts':[17754861,399998,np.nan]},index=[0,1,2])

In [10]:
mapping_products=pd.read_table("/home/jian/BigLots/static_files/ProductTaxonomy/MediaStormProductTaxonomy20190501-135142-836.txt",
                               dtype=str,sep="|")
mapping_products=mapping_products[['division_id','class_code_id','subclass_id']].drop_duplicates()

In [11]:
control_400K=pd.read_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_20190107/Pick_Control_within_Test/output_control_with_in_Q1_test.csv",dtype=str)

control_400K.shape

(399998, 3)

In [12]:
test_test_Q1=glob.glob("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_20190107/by_group_quadrant_24/*.csv")
test_test_Q1=[x for x in test_test_Q1 if "/T_Quadrant" in x]
all_test=pd.DataFrame()
for file_path in test_test_Q1:
    df=pd.read_csv(file_path,dtype=str)
    all_test=all_test.append(df)
print(all_test['customer_id_hashed'].shape)   
all_test=all_test[~all_test['customer_id_hashed'].isin(control_400K['customer_id_hashed'].tolist())]
print(all_test['customer_id_hashed'].shape)
all_test=all_test[~all_test['email_address_hash'].isin(control_400K['email_address_hash'].tolist())]
print(all_test['customer_id_hashed'].shape)

(18154859,)
(17754861,)
(17754861,)


In [13]:
control_400K=control_400K[['customer_id_hashed']]
control_400K['group']='control_400K'

all_test=all_test[['customer_id_hashed']]
all_test['group']='test_in_test'

df_all_id_Q1_test=all_test.append(control_400K)
print(df_all_id_Q1_test.shape)
print(df_all_id_Q1_test['customer_id_hashed'].nunique())


(18154859, 2)
18154859


In [14]:
daily_file_list=list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
daily_file_list=[x for x in daily_file_list if "aily" in x]
df_daily_files=pd.DataFrame({'file_path':daily_file_list})
df_daily_files['week_end_dt']=df_daily_files['file_path'].apply(lambda x: x.split("Lots/2019_by_weeks/MediaStorm_")[1][:10])

df_daily_files=df_daily_files[(df_daily_files['week_end_dt']>="2019-03-17") & (df_daily_files['week_end_dt']<="2019-05-25")]
print(df_daily_files.shape)
print(df_daily_files['week_end_dt'].nunique())

(10, 2)
10


In [16]:
df_output_all=pd.DataFrame()
df_output_division=pd.DataFrame()

for daily_file in df_daily_files['file_path'].unique():
    df=pd.read_table(daily_file,dtype=str,sep="|",
                     usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed','class_code_id','subclass_id','item_transaction_units','item_transaction_amt'])
    df=df[pd.notnull(df['customer_id_hashed'])]
    df['transaction_dt']=pd.to_datetime(df['transaction_dt'],format="%Y-%m-%d")
    
    df=pd.merge(df,mapping_products,on=['class_code_id','subclass_id'],how="left")
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df['item_transaction_units']=df['item_transaction_units'].astype(int)
    df['Ecommerce_group']=np.where(df['location_id']=="6990","Online","Instore")
    
    df_sales=df.groupby(['customer_id_hashed','transaction_dt','Ecommerce_group'])['item_transaction_amt','item_transaction_units'].sum().reset_index().rename(columns={"item_transaction_amt":"sales","item_transaction_units":"units"})
    df_trans=df[['location_id','transaction_dt','transaction_id','customer_id_hashed','Ecommerce_group']].drop_duplicates()
    df_trans=df_trans.groupby(['customer_id_hashed','transaction_dt','Ecommerce_group'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"trans"})   
    df_all_products=pd.merge(df_sales,df_trans,on=['customer_id_hashed','transaction_dt','Ecommerce_group'],how="outer")
    
    df_sales_division=df.groupby(['customer_id_hashed','transaction_dt','Ecommerce_group','division_id'])['item_transaction_amt','item_transaction_units'].sum().reset_index().rename(columns={"item_transaction_amt":"sales","item_transaction_units":"units"})
    df_trans_division=df[['location_id','transaction_dt','transaction_id','customer_id_hashed','Ecommerce_group','division_id']].drop_duplicates()
    df_trans_division=df_trans_division.groupby(['customer_id_hashed','transaction_dt','Ecommerce_group','division_id'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"trans"})
    df_division=pd.merge(df_sales_division,df_trans_division,on=['customer_id_hashed','Ecommerce_group','transaction_dt','division_id'],how="outer")
    
    df_output_all=df_output_all.append(df_all_products)
    df_output_division=df_output_division.append(df_division)
    
    print(datetime.datetime.now(),df.shape)
    

2019-05-30 10:37:21.352096 (7804184, 10)
2019-05-30 10:42:05.672978 (7697772, 10)
2019-05-30 10:47:38.274888 (10699521, 10)
2019-05-30 10:52:36.035831 (9827794, 10)
2019-05-30 10:56:20.913494 (8254651, 10)
2019-05-30 10:59:24.088496 (6851914, 10)
2019-05-30 11:02:34.463618 (8053397, 10)
2019-05-30 11:05:25.921088 (7309477, 10)
2019-05-30 11:08:30.402792 (7042143, 10)
2019-05-30 11:12:40.415353 (7905943, 10)


In [17]:
df_output_division=pd.merge(df_output_division,df_all_id_Q1_test,on="customer_id_hashed",how="left")
df_output_all=pd.merge(df_output_all,df_all_id_Q1_test,on="customer_id_hashed",how="left")

df_output_division['group']=df_output_division['group'].fillna("balance")
df_output_all['group']=df_output_all['group'].fillna("balance")

In [18]:
def week_start_Wednesday(x):
    if x.weekday()>=2:
        y=(x-datetime.timedelta(days=(x.weekday()-2))).date()
    else:
        y=(x-datetime.timedelta(days=(x.weekday()+7-2))).date()
        
    return y

print(datetime.datetime.now())
df_output_all['week_start_Wednesday']=df_output_all['transaction_dt'].apply(lambda x: week_start_Wednesday(x))
print(datetime.datetime.now())
df_output_division['week_start_Wednesday']=df_output_division['transaction_dt'].apply(lambda x: week_start_Wednesday(x))
print(datetime.datetime.now())

2019-05-30 11:14:59.165035
2019-05-30 11:17:13.932145
2019-05-30 11:22:55.325814


In [19]:
df_output_all=pd.merge(df_output_all,df_week_start_group,on="week_start_Wednesday",how="left")
df_output_division=pd.merge(df_output_division,df_week_start_group,on="week_start_Wednesday",how="left")

In [22]:
df_output_all.head(2)

,customer_id_hashed,transaction_dt,Ecommerce_group,sales,units,trans,group,week_start_Wednesday,week_period
0,000007eccbfdc528a745aea90d34404b4fae9fbad2525f...,2019-03-22,Instore,15.44,9,1,balance,2019-03-20,P1
1,000015cec4564c97c324f658df6de41d8df29d0858826b...,2019-03-20,Instore,8.00,1,1,test_in_test,2019-03-20,P1


In [25]:
df_output_division.head(2)

,customer_id_hashed,transaction_dt,Ecommerce_group,division_id,sales,units,trans,group,week_start_Wednesday,week_period
0,000007eccbfdc528a745aea90d34404b4fae9fbad2525f...,2019-03-22,Instore,1,2.79,2,1,balance,2019-03-20,P1
1,000007eccbfdc528a745aea90d34404b4fae9fbad2525f...,2019-03-22,Instore,2,6.75,3,1,balance,2019-03-20,P1


In [24]:
# 2018 data to ask if needed because that the audience may not be available, 1 year prior of the scoring

df_output_all.to_csv('/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Incremental_1mm_test/BL_df_overall_2019Q1_400K_test_JL_'+str(datetime.datetime.now().date())+".csv",index=False)
df_output_division.to_csv('/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Incremental_1mm_test/BL_df_division_2019Q1_400K_test_JL_'+str(datetime.datetime.now().date())+".csv",index=False)




In [33]:
df_output_all=pd.read_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Incremental_1mm_test/BL_df_overall_2019Q1_400K_test_JL_2019-05-30.csv",dtype=str)
df_output_division=pd.read_csv('/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Incremental_1mm_test/BL_df_division_2019Q1_400K_test_JL_2019-05-30.csv',dtype=str)

df_output_all['sales']=df_output_all['sales'].astype(float)
df_output_all['units']=df_output_all['units'].astype(int)
df_output_all['trans']=df_output_all['trans'].astype(int)


df_output_division['sales']=df_output_division['sales'].astype(float)
df_output_division['units']=df_output_division['units'].astype(int)
df_output_division['trans']=df_output_division['trans'].astype(int)

In [34]:
df_output_all_Online=df_output_all[df_output_all['Ecommerce_group']=="Online"].rename(columns={"sales":"Online_Sales","units":"Online_Units","trans":"Online_Trans"})
df_output_all_Instore=df_output_all[df_output_all['Ecommerce_group']=="Instore"].rename(columns={"sales":"Instore_Sales","units":"Instore_Units","trans":"Instore_Trans"})
del df_output_all_Online['Ecommerce_group']
del df_output_all_Instore['Ecommerce_group']

df_output_all=pd.merge(df_output_all_Online,df_output_all_Instore,on=['customer_id_hashed','transaction_dt','group','week_start_Wednesday','week_period'],how="outer")
df_output_all=df_output_all.fillna(0)


In [35]:
df_output_division_Online=df_output_division[df_output_division['Ecommerce_group']=="Online"].rename(columns={"sales":"Online_Sales","units":"Online_Units","trans":"Online_Trans"})
df_output_division_Instore=df_output_division[df_output_division['Ecommerce_group']=="Instore"].rename(columns={"sales":"Instore_Sales","units":"Instore_Units","trans":"Instore_Trans"})
del df_output_division_Online['Ecommerce_group']
del df_output_division_Instore['Ecommerce_group']

df_output_division=pd.merge(df_output_division_Online,df_output_division_Instore,on=['customer_id_hashed','transaction_dt','group','week_start_Wednesday','week_period','division_id'],how="outer")
df_output_division=df_output_division.fillna(0)

In [46]:
del df_output_all_Online
del df_output_all_Instore
del df_output_division_Online
del df_output_division_Instore
gc.collect()

4750

In [38]:
df_output_all=df_output_all[df_output_all['week_period']!=0]
df_output_division=df_output_division[df_output_division['week_period']!=0]

In [39]:
func_dict_apply={'customer_id_hashed':'nunique',
                 'Online_Sales':'sum',
                 'Online_Units':'sum',
                 'Online_Trans':'sum', 
                 'Instore_Sales':'sum',
                 'Instore_Units':'sum',
                 'Instore_Trans':'sum',
                }

In [50]:
# 1 week,group
output_table_1=df_output_all.groupby(['group','week_start_Wednesday','week_period']).agg(func_dict_apply).reset_index()
# 2 week,group,division
output_table_2=df_output_division.groupby(['group','week_start_Wednesday','week_period','division_id']).agg(func_dict_apply).reset_index()

# 3 period,group
output_table_3=df_output_all.groupby(['group','week_period']).agg(func_dict_apply).reset_index()
# 4 period,group,division
output_table_4=df_output_division.groupby(['group','week_period','division_id']).agg(func_dict_apply).reset_index()


In [51]:
df_divison_name=pd.read_csv("/home/jian/BigLots/static_files/MediaStorm Data Extract - Division Names.txt",dtype=str,sep="|")
output_table_2=pd.merge(output_table_2,df_divison_name,on="division_id",how="left")
output_table_4=pd.merge(output_table_4,df_divison_name,on="division_id",how="left")

In [53]:
output_table_1=pd.merge(output_table_1,df_group_total_counts,on="group",how="left")
output_table_2=pd.merge(output_table_2,df_group_total_counts,on="group",how="left")
output_table_3=pd.merge(output_table_3,df_group_total_counts,on="group",how="left")
output_table_4=pd.merge(output_table_4,df_group_total_counts,on="group",how="left")

In [54]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Incremental_1mm_test/BL_output_2019Q1_400K_test_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
output_table_1.to_excel(writer,"by_group_week",index=False)
output_table_2.to_excel(writer,"by_group_week_division",index=False)
output_table_3.to_excel(writer,"by_group_period",index=False)
output_table_4.to_excel(writer,"by_group_period_division",index=False)
writer.save()

In [20]:
df_output_all.groupby(['group'])['customer_id_hashed'].nunique().to_frame().reset_index()

,group,customer_id_hashed
0,balance,2040787
1,control_400K,115302
2,test_in_test,5122366


In [21]:
df_output_division.groupby(['group'])['customer_id_hashed'].nunique().to_frame().reset_index()

,group,customer_id_hashed
0,balance,2040787
1,control_400K,115302
2,test_in_test,5122366


,group,total_counts
0,test_in_test,17754861.0
1,control_400K,399998.0
2,balance,NaN


In [56]:
df_week_start_group

,week_start_Wednesday,week_period
0,2019-03-20,P1
1,2019-03-27,P1
2,2019-04-03,P1
3,2019-04-10,P1
4,2019-04-17,P2
5,2019-04-24,P3
6,2019-05-01,P4
7,2019-05-08,P4
8,2019-05-15,P4
